# Sign Language Gesture Recognition

CNN-based recognition of 24 static ASL hand gestures (A-I, K-Y) trained on Sign Language MNIST.

**Pipeline:**
1. Dataset loading
2. Data preprocessing & augmentation
3. CNN model architecture
4. Training
5. Evaluation (accuracy, confusion matrix, learning curve)
6. Prediction on sample images

## 1. Setup & Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization,
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 2. Load Dataset

Upload `sign_mnist_train.csv` and `sign_mnist_test.csv` from [Kaggle](https://www.kaggle.com/datasets/datamunge/sign-language-mnist).

In [ ]:
from google.colab import files

print("Upload sign_mnist_train.csv and sign_mnist_test.csv")
uploaded = files.upload()

In [ ]:
# Constants
IMG_SIZE = 28
NUM_CLASSES = 25
LABEL_TO_LETTER = {i: chr(ord('A') + i + (1 if i >= 9 else 0)) for i in range(25)}
LETTER_TO_LABEL = {v: k for k, v in LABEL_TO_LETTER.items()}

def load_csv(filepath):
    """Load a Sign Language MNIST CSV. Returns (images, labels)."""
    df = pd.read_csv(filepath)
    labels = df.iloc[:, 0].values.astype(np.int32)
    images = df.iloc[:, 1:].values.astype(np.float32).reshape(-1, IMG_SIZE, IMG_SIZE)
    return images, labels

train_images, train_labels = load_csv("sign_mnist_train.csv")
test_images, test_labels = load_csv("sign_mnist_test.csv")

print(f"Training samples: {len(train_images)}")
print(f"Test samples:     {len(test_images)}")
print(f"Image shape:      {train_images[0].shape}")
print(f"Label range:      {train_labels.min()} - {train_labels.max()}")
print(f"Unique classes:   {len(np.unique(train_labels))}")

## 3. Explore Dataset

In [ ]:
# Class distribution
fig, ax = plt.subplots(figsize=(12, 4))
unique, counts = np.unique(train_labels, return_counts=True)
letters = [LABEL_TO_LETTER[u] for u in unique]
ax.bar(letters, counts, color="steelblue")
ax.set_xlabel("Letter")
ax.set_ylabel("Count")
ax.set_title("Training Set Class Distribution")
plt.tight_layout()
plt.show()

In [ ]:
# Sample images
fig, axes = plt.subplots(3, 8, figsize=(14, 6))
for i, ax in enumerate(axes.flat):
    idx = np.random.randint(len(train_images))
    ax.imshow(train_images[idx], cmap="gray")
    ax.set_title(LABEL_TO_LETTER[train_labels[idx]], fontsize=12)
    ax.axis("off")
fig.suptitle("Sample Training Images", fontsize=14)
plt.tight_layout()
plt.show()

## 4. Data Preprocessing

In [ ]:
def preprocess_pipeline(images, labels):
    """Normalize to [0,1], reshape to (N,28,28,1), one-hot encode labels."""
    images = images.astype(np.float32) / 255.0
    images = images.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    labels = to_categorical(labels, num_classes=NUM_CLASSES)
    return images, labels

X_train_full, y_train_full = preprocess_pipeline(train_images, train_labels)
X_test, y_test = preprocess_pipeline(test_images, test_labels)

# Train / validation split (90/10)
val_split = 0.1
num_val = int(len(X_train_full) * val_split)
idx = np.random.permutation(len(X_train_full))

X_val, y_val = X_train_full[idx[:num_val]], y_train_full[idx[:num_val]]
X_train, y_train = X_train_full[idx[num_val:]], y_train_full[idx[num_val:]]

print(f"Train: {len(X_train)}  Val: {len(X_val)}  Test: {len(X_test)}")

## 5. Data Augmentation

In [ ]:
augmenter = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
)
augmenter.fit(X_train)

# Visualize augmented samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
sample = X_train[0:1]
for ax in axes.flat:
    aug_img = augmenter.random_transform(sample[0])
    ax.imshow(aug_img.squeeze(), cmap="gray")
    ax.axis("off")
fig.suptitle("Augmented Samples (same source image)", fontsize=14)
plt.tight_layout()
plt.show()

## 6. CNN Model Architecture

In [ ]:
def build_cnn():
    model = Sequential([
        Conv2D(32, 3, activation="relu", padding="same", input_shape=(IMG_SIZE, IMG_SIZE, 1)),
        BatchNormalization(),
        Conv2D(32, 3, activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D(2),
        Dropout(0.25),

        Conv2D(64, 3, activation="relu", padding="same"),
        BatchNormalization(),
        Conv2D(64, 3, activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D(2),
        Dropout(0.25),

        Flatten(),
        Dense(256, activation="relu"),
        BatchNormalization(),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation="softmax"),
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

model = build_cnn()
model.summary()

## 7. Training

In [ ]:
EPOCHS = 30
BATCH_SIZE = 64

callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
]

history = model.fit(
    augmenter.flow(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1,
)

## 8. Learning Curve

In [ ]:
h = history.history
epochs = range(1, len(h["accuracy"]) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
ax1.plot(epochs, h["accuracy"], "b-o", markersize=4, label="Training")
ax1.plot(epochs, h["val_accuracy"], "r-o", markersize=4, label="Validation")
ax1.set_title("Model Accuracy", fontsize=14)
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Accuracy")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Loss
ax2.plot(epochs, h["loss"], "b-o", markersize=4, label="Training")
ax2.plot(epochs, h["val_loss"], "r-o", markersize=4, label="Validation")
ax2.set_title("Model Loss", fontsize=14)
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Loss")
ax2.legend()
ax2.grid(True, alpha=0.3)

fig.suptitle("Learning Curve", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.show()

## 9. Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f} ({test_acc * 100:.2f}%)")
print(f"Test Loss:     {test_loss:.4f}")

In [ ]:
# Confusion matrix & per-class metrics
y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
y_true = np.argmax(y_test, axis=1)
class_names = [LABEL_TO_LETTER[i] for i in range(NUM_CLASSES)]

cm = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=np.int64)
for t, p in zip(y_true, y_pred):
    cm[t, p] += 1

# Per-class metrics
print(f"\n{'':>6s} {'prec':>6s} {'rec':>6s} {'f1':>6s} {'n':>6s}")
print("-" * 32)
for i in range(NUM_CLASSES):
    tp = cm[i, i]
    fp = cm[:, i].sum() - tp
    fn = cm[i, :].sum() - tp
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
    print(f"{class_names[i]:>6s} {prec:>6.3f} {rec:>6.3f} {f1:>6.3f} {int(cm[i].sum()):>6d}")

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(14, 12))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names, linewidths=0.5)
plt.title("Confusion Matrix", fontsize=16)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

## 10. Sample Predictions

In [ ]:
fig, axes = plt.subplots(3, 6, figsize=(14, 7))
indices = np.random.choice(len(X_test), 18, replace=False)

for ax, idx in zip(axes.flat, indices):
    pred = y_pred[idx]
    true = y_true[idx]
    ax.imshow(X_test[idx].squeeze(), cmap="gray")
    color = "green" if pred == true else "red"
    ax.set_title(f"P:{LABEL_TO_LETTER[pred]} T:{LABEL_TO_LETTER[true]}", color=color, fontsize=11)
    ax.axis("off")

fig.suptitle("Sample Predictions (green=correct, red=wrong)", fontsize=14)
plt.tight_layout()
plt.show()

## 11. Save Model

In [ ]:
model.save("trained_model.h5")
print("Model saved to trained_model.h5")

# Download the model file
files.download("trained_model.h5")